In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install -q kaggle

!mkdir ~/.kaggle
!cp "/content/drive/MyDrive/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download "stefanoleone992/fifa-23-complete-player-dataset"

!unzip -q fifa-23-complete-player-dataset.zip -d "fifa23"

!mv ./fifa23/male_players.csv ./
!mv "/content/fifa23/male_players (legacy).csv" ./

!rm -rf fifa23
!rm ./fifa-23-complete-player-dataset.zip


 99% 1.56G/1.58G [00:15<00:00, 36.8MB/s]
100% 1.58G/1.58G [00:15<00:00, 112MB/s] 


In [ ]:
import numpy as np
import pandas as pd
import dask.dataframe as dd

In [ ]:
data = pd.read_csv("./male_players (legacy).csv")

data.head()

<ipython-input-118-2066fff0c603>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./male_players (legacy).csv")


,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [ ]:
data['attacking_mean'] = np.mean(data[['attacking_crossing',
                                       'attacking_finishing',
                                       'attacking_heading_accuracy',
                                       'attacking_short_passing',
                                       'attacking_volleys']], axis=1)

data['skill_mean'] = np.mean(data[['skill_dribbling',
                                   'skill_curve',
                                   'skill_fk_accuracy',
                                   'skill_long_passing',
                                   'skill_ball_control']], axis=1)

data['movement_mean'] = np.mean(data[['movement_acceleration',
                                      'movement_sprint_speed',
                                      'movement_agility',
                                      'movement_reactions',
                                      'movement_balance']], axis=1)

data['power_mean'] = np.mean(data[['power_shot_power',
                                   'power_jumping',
                                   'power_stamina',
                                   'power_strength',
                                   'power_long_shots']], axis=1)

data['mentality_mean'] = np.mean(data[['mentality_aggression',
                                       'mentality_interceptions',
                                       'mentality_positioning',
                                       'mentality_vision',
                                       'mentality_penalties',
                                       'mentality_composure']], axis=1)

data['defending_mean'] = np.mean(data[['defending_marking_awareness',
                                       'defending_standing_tackle',
                                       'defending_sliding_tackle']], axis=1)

data['goalkeeping_mean'] = np.mean(data[['goalkeeping_diving',
                                         'goalkeeping_handling',
                                         'goalkeeping_kicking',
                                         'goalkeeping_positioning',
                                         'goalkeeping_reflexes',
                                         'goalkeeping_speed']], axis=1)


In [ ]:
columns_to_keep = [
                  'overall',
                  'potential',
                  'value_eur',
                  'wage_eur',
                  'age',
                  'height_cm',
                  'weight_kg',
                  'league_id',
                  'league_level',
                  'club_contract_valid_until_year',
                  'preferred_foot',
                  'weak_foot',
                  'skill_moves',
                  'international_reputation',
                  'work_rate',
                  'body_type',
                  'release_clause_eur',
                  'pace',
                  'shooting',
                  'passing',
                  'dribbling',
                  'defending',
                  'physic',
                  'attacking_mean',
                  'skill_mean',
                  'movement_mean',
                  'power_mean',
                  'mentality_mean',
                  'defending_mean',
                  'goalkeeping_mean']
columns_to_delete = list(set(data.columns.to_list()) - set(columns_to_keep))
data = data.drop(columns=columns_to_delete)
data.head()

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_level,club_contract_valid_until_year,...,dribbling,defending,physic,attacking_mean,skill_mean,movement_mean,power_mean,mentality_mean,defending_mean,goalkeeping_mean
0,93,95,100500000.00,550000.00,27,169,67,53.00,1.00,2018.00,...,96.00,27.00,63.00,84.60,89.40,93.80,75.60,65.60,22.00,10.80
1,92,92,79000000.00,375000.00,29,185,80,53.00,1.00,2018.00,...,91.00,32.00,79.00,86.60,84.80,86.20,89.80,68.80,25.33,11.60
2,90,90,54500000.00,275000.00,30,180,80,19.00,1.00,2017.00,...,92.00,32.00,64.00,77.40,85.40,91.80,76.00,67.80,27.00,9.80
3,90,90,52500000.00,275000.00,32,195,95,16.00,1.00,2016.00,...,86.00,34.00,86.00,83.80,82.80,72.60,84.80,72.80,31.00,11.80
4,90,90,63500000.00,300000.00,28,193,92,19.00,1.00,2019.00,...,NaN,NaN,NaN,28.40,29.40,57.20,54.40,28.20,25.00,83.33


In [ ]:
data.shape

(161583, 30)

In [ ]:
data = data.dropna()

In [ ]:
data.shape

(91567, 30)

In [ ]:
data.dtypes

overall                             int64
potential                           int64
value_eur                         float64
wage_eur                          float64
age                                 int64
height_cm                           int64
weight_kg                           int64
league_id                         float64
league_level                      float64
club_contract_valid_until_year    float64
preferred_foot                     object
weak_foot                           int64
skill_moves                         int64
international_reputation            int64
work_rate                          object
body_type                          object
release_clause_eur                float64
pace                              float64
shooting                          float64
passing                           float64
dribbling                         float64
defending                         float64
physic                            float64
attacking_mean                    

In [ ]:
data[['preferred_foot','work_rate', 'body_type']]

,preferred_foot,work_rate,body_type
50427,Right,High/Low,Unique
50428,Left,Medium/Medium,Unique
50430,Right,High/Medium,Normal (170-185)
50431,Right,High/Medium,Unique
50432,Right,High/Medium,Normal (170-185)
...,...,...,...
161578,Right,Medium/Medium,Lean (170-185)
161579,Left,Medium/Medium,Normal (185+)
161580,Right,High/Medium,Normal (170-185)
161581,Right,Medium/Medium,Normal (170-185)


In [ ]:
data['preferred_foot'] = (data['preferred_foot'] == 'Left').astype(int)

In [ ]:
data = pd.get_dummies(data, columns=['work_rate', 'body_type'])

In [ ]:
data.dtypes

overall                             int64
potential                           int64
value_eur                         float64
wage_eur                          float64
age                                 int64
height_cm                           int64
weight_kg                           int64
league_id                         float64
league_level                      float64
club_contract_valid_until_year    float64
preferred_foot                      int64
weak_foot                           int64
skill_moves                         int64
international_reputation            int64
release_clause_eur                float64
pace                              float64
shooting                          float64
passing                           float64
dribbling                         float64
defending                         float64
physic                            float64
attacking_mean                    float64
skill_mean                        float64
movement_mean                     

In [ ]:
data[data['player_id']==158023]

KeyError: ignored

In [ ]:
from sklearn.preprocessing import StandardScaler

columns = data.columns.to_list()

scaler = StandardScaler()
data[columns] = scaler.fit_transform(data[columns])

data.head()

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_level,club_contract_valid_until_year,...,body_type_Lean (170-),body_type_Lean (170-185),body_type_Lean (185+),body_type_Normal (170-),body_type_Normal (170-185),body_type_Normal (185+),body_type_Stocky (170-),body_type_Stocky (170-185),body_type_Stocky (185+),body_type_Unique
50427,4.04,3.75,13.38,26.17,1.48,0.71,0.85,-0.39,-0.50,-0.30,...,-0.17,-0.56,-0.32,-0.20,-0.77,-0.45,-0.08,-0.19,-0.13,15.96
50428,3.90,3.58,14.75,26.17,1.05,-1.61,-0.35,-0.39,-0.50,-1.81,...,-0.17,-0.56,-0.32,-0.20,-0.77,-0.45,-0.08,-0.19,-0.13,15.96
50430,3.76,3.42,13.60,22.69,1.05,0.24,1.74,-0.39,-0.50,-0.30,...,-0.17,-0.56,-0.32,-0.20,1.30,-0.45,-0.08,-0.19,-0.13,-0.06
50431,3.76,3.75,17.35,12.27,-0.04,-0.83,-0.95,-0.46,-0.50,0.20,...,-0.17,-0.56,-0.32,-0.20,-0.77,-0.45,-0.08,-0.19,-0.13,15.96
50432,3.61,3.25,12.88,15.75,0.61,0.71,0.70,-0.46,-0.50,-0.30,...,-0.17,-0.56,-0.32,-0.20,1.30,-0.45,-0.08,-0.19,-0.13,-0.06


STEPS TO PERFORM

1. ~~REMOVE NA CELLS~~
2. ~~REMOVE REDUDANT COLUMNS~~
3. ~~DIVIDE THE COLUMNS IN X AND y~~
4. ~~train_test_split~~
5. TRAIN LINEAR REGRESSION MODEL
6. FIND OUT THE ACCURACY AND R2


Need to check whether removing the player id, fifa version and fifa update columns help improve the accuracy..

- before scaling

  - without removing:
    - Mean Squared Error: 1587133444056.8484
    - Root Mean Squared Error: 1259814.8451486228
    - R-squared: 0.9622695722399305

  - after removing:
    - Mean Squared Error: 1588167356410.5562
    - Root Mean Squared Error: 1260225.1213218041
    - R-squared: 0.9622449933644

- after scaling:
  - Mean Squared Error: 0.03307821645198476
  - Root Mean Squared Error: 0.1818741775293699
  - R-squared: 0.9622449933644271


In [ ]:
y = data["value_eur"]
X = data.drop("value_eur", axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

Mean Squared Error: 0.03307821645198476
Root Mean Squared Error: 0.1818741775293699
R-squared: 0.9622449933644271


In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.DataFrame({"overall": X_test['overall'], "test_value_eur": y_test, "Pred_value_eur": y_pred})


,overall,test_value_eur,Pred_value_eur
84627,-1.31,-0.37,-0.38
156628,-0.59,-0.26,-0.22
56380,0.42,-0.35,-0.35
135646,-0.30,-0.21,-0.15
132368,0.14,-0.27,-0.28
...,...,...,...
87996,1.44,0.61,0.62
99972,-0.59,-0.36,-0.35
92884,0.42,-0.17,-0.08
74794,0.42,-0.15,-0.11
